# Part 1.8. MNIST Introduction

In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import random

## 1. Device Setting

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# seed 설정
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

## 2. Dataset
* `pytorch` : 0과 1사이의 값으로 (Channel, Height, Width) 로 구성
* `img` : 0과 255사이의 값으로 (Height, Width, Channel)로 구성

In [3]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),    # pytorch에 맞게 변형
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


In [0]:
# parameters
training_epochs = 15
batch_size = 100

In [0]:
# 학습시킬 때 데이터를 불러올 때 배치 크기만큼 편하게 잘라줌
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,    # 사용할 데이터셋
                                          batch_size=batch_size,  # 배치 사이즈
                                          shuffle=True,           # 불러올 때마다 데이터를 섞음
                                          drop_last=True)         # 배치 크기만큼 자를 때 나누어 떨어지지 않으면 버림

## 3. Model

In [0]:
linear = torch.nn.Linear(784, 10, bias=True).to(device)     # 입력 784, 라벨 10, bias  사용

In [0]:
criterion = torch.nn.CrossEntropyLoss().to(device)          # cost function 설정
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)    # optimizer 설정

## 4. Train
* `epoch` : 학습 데이터 전체를 몇 번 도는지. 전체를 한 번 돌면 1 epoch를 돌았다라고 함.
* `batch size` : 학습 시에 학습할 데이터의 양. 
  * 예를 들어 배치 크기가 100이면 한 번 학습 시 100개의 데이터를 이용함. 
  * 한 번 forward, backward할 때 쓰이는 데이터의 양을 말함.
* `iteration` : batch를 몇 번 학습에 이용했는지.

In [8]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)

    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)    # (batchsize, 784)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.535468519
Epoch: 0002 cost = 0.359274149
Epoch: 0003 cost = 0.331187546
Epoch: 0004 cost = 0.316578060
Epoch: 0005 cost = 0.307158172
Epoch: 0006 cost = 0.300180733
Epoch: 0007 cost = 0.295130223
Epoch: 0008 cost = 0.290851504
Epoch: 0009 cost = 0.287417054
Epoch: 0010 cost = 0.284379601
Epoch: 0011 cost = 0.281825215
Epoch: 0012 cost = 0.279800683
Epoch: 0013 cost = 0.277808994
Epoch: 0014 cost = 0.276154310
Epoch: 0015 cost = 0.274440825
Learning finished


## 5. Test

In [9]:
with torch.no_grad():   # 기울기 계산을 하지 않음 (실수 방지용)
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.8863000273704529


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
